In [ ]:
from pathlib import Path
from typing import List, Tuple
import argparse
import pandas as pd
import numpy as np
import pyvista as pv

from src.config import GridSpec
from src.voxel import Voxeliser
from src.plotter import VoxelPlotter

In [ ]:
parquet_file = "data/cluster_2001-2005.parquet"
vox_sz = 1.0
dt = 1.0

print(f"Reading {parquet_file}")
df = pd.read_parquet(parquet_file)
grid = GridSpec(voxel=vox_sz)
for ax in ("x", "y", "z"):
    df[ax] /= grid.re_km        # km -> RE

vox = Voxeliser(grid)
vox.accumulate(df, dt_minutes=dt)

In [ ]:
regions, P = vox.probs()               # P[nx,ny,nz,R]
reg = "IN/PSH"
idx = regions.index(reg)

z0_phys = 0.0                            # z in RE
z0_idx = np.digitize([z0_phys], grid.edges) - 1
z0_idx = int(z0_idx[0])

import matplotlib.pyplot as plt
plt.imshow(P[:, :, z0_idx, idx].T, origin="lower",
           extent=[grid.edges[0], grid.edges[-1],
                   grid.edges[0], grid.edges[-1]],
           cmap="inferno", vmin=0, vmax=1.0)
plt.colorbar(label=f"P({reg})")
plt.xlabel("X  [R$_E$]")
plt.ylabel("Y  [R$_E$]")
plt.title(f"{reg} probability at Z={z0_phys} R$_E$")
plt.gca().set_aspect("equal")
plt.show()


In [ ]:
# ------------------------------------------------------------------
# 2-D region map (mode region) at Z = 0  R_E
# ------------------------------------------------------------------
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap



# --- winner region per (x,y) --------------------------------------
slice_probs = P[:, :, z0_idx, :]                # (nx, ny, R)
winner_idx   = slice_probs.argmax(axis=-1)  # region index of max P
total_prob   = slice_probs.sum(axis=-1)     # 0 where voxel never visited
winner_idx[total_prob == 0] = -1            # mark empty voxels

# --- discrete colour table ---------------------------------------
region_colors = {
    # IN/ regions (deep -> light blue)
    "IN/UKN":   "#8FBFFF",   # light blue
    "IN/PLS":   "#6EA8FF",
    "IN/PPTR":  "#4A90FF",
    "IN/PSH":   "#256BFF",
    "IN/PSTR":  "#004CFF",
    "IN/LOB":   "#003399",   # deep blue
    "IN/POL":   "#001D66",   # darkest
    # Boundary (magnetopause)
    "IN/MP":    "#A060E8",   # purple
    "IN/MPTR":  "#C184F5",
    # OUT/ regions (orange-red range)
    "OUT/MSH":  "#FF3D3D",   # strong red
    "OUT/BSTR": "#FF6A3D",   # reddish-orange
    "OUT/SWF":  "#FF944D",   # orange
    "OUT/UKN":  "#FFB570",   # light orange
    # Non-physical or unknown
    "UNKNOWN":  "#9E9E9E",   # medium grey
    "N/A":      "#D3D3D3",   # light grey
}

# Build a list aligned with `regions` plus one for "empty"
color_list = ["black"] + [region_colors.get(r, "#FFFFFF") for r in regions]
cmap = ListedColormap(color_list)

# Add +1 so empty(-1)->0, first region->1, ...
plot_data = winner_idx + 1

# --- plot ---------------------------------------------------------
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(plot_data.T, origin="lower",
               extent=[grid.edges[0], grid.edges[-1],
                       grid.edges[0], grid.edges[-1]],
               cmap=cmap, vmin=0, vmax=len(color_list)-1)

# Draw Earth at the origin
earth = mpatches.Circle((0, 0), radius=1.0, color="lightblue", zorder=10)
ax.add_patch(earth)

ax.set_xlabel("X  [R$_E$]")
ax.set_ylabel("Y  [R$_E$]")
ax.set_title(f"Dominant magnetospheric region  (Z = {z0_phys} R$_E$)")
ax.set_aspect("equal")

# --- legend -------------------------------------------------------
handles = [mpatches.Patch(color="black", label=r"$\emptyset$ (no data)"),
           mpatches.Patch(color="lightblue", label="Earth (1 $R_E$)")]
for r, col in region_colors.items():
    if r in regions:
        handles.append(mpatches.Patch(color=col, label=r))
ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.show()
